In [55]:
%pip install selenium webdriver-manager
%pip install geopy folium mapclassify

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 9.5 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 8.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 KB 8.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [42]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service as ChromeService 
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.chrome.options import Options
import pandas as pd
import geopandas as gpd

In [57]:
url_to_scrape = "https://www.therapie.de/therapeutensuche/ergebnisse/?abrechnungsverfahren=1&arbeitsschwerpunkt=9&ort=50735&terminzeitraum=2&therapieangebot=1&verfahren=2"

'''chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--headless=new")
chrome_options.add_argument("--remote-debugging-port=9222")

with webdriver.Chrome(options=chrome_options) as driver:

    driver.get(url_to_scrape)

    innerHTML = driver.execute_script("return document.body.innerHTML")

    html_soup = BeautifulSoup(
        innerHTML,
        'html.parser'
    )'''

html_soup = BeautifulSoup(
    urlopen(url_to_scrape),
    'html.parser'
)

In [58]:
print(html_soup.prettify())

<!DOCTYPE HTML>
<html lang="de">
 <head>
  <meta content="ie=edge" http-equiv="x-ua-compatible"/>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <!-- head:dynamic -->
  <title>
   21 Therapeuten im Umkreis von 50735 | therapie.de
  </title>
  <meta content="Psychotherapie gesucht? Finden Sie Ihren Psychotherapeuten, Arzt, Psychologen oder Heilpraktiker für Psychotherapie" name="description"/>
  <meta content="" name="keywords"/>
  <meta content="INDEX,NOFOLLOW" name="robots"/>
  <link href="https://www.therapie.de/therapeutensuche/ergebnisse/?abrechnungsverfahren=1&amp;arbeitsschwerpunkt=9&amp;ort=50735&amp;page=2&amp;terminzeitraum=2&amp;therapieangebot=1&amp;verfahren=2" rel="next"/>
  <script type="application/ld+json">
   { "@context" : "http://schema.org", "@type" : "WebSite", "name" : "therapie.de", "url" : "https://www.therapie.de" }
  </script>
  <link href="/favicon.ico?v=3" rel="shortcut icon" type="image/x-icon">
   <link 

In [59]:
base_url = 'https://www.therapie.de'

names = []
addresses = []
links = []
geometries = []

for result in html_soup.find('div', 'search-results').find_all('a', href=True):
    names.append(result.find("div", 'search-results-name').text)
    address_and_phone = result.find("div", 'search-results-address').get_text(strip=True, separator='\n').splitlines()[1]
    address = address_and_phone.rsplit(', ', 1)[0]
    addresses.append(address)
    links.append(base_url + result['href'])
    geometries.append(gpd.tools.geocode(address).geometry[0])

data = gpd.GeoDataFrame({
        'name': names,
        'address': addresses,
        'link': links,
        'geometry': geometries,
    },
    crs="EPSG:4326",
)

data

,name,address,link,geometry
0,Dipl.-Psych. Frank Nachtigall,"Neusser Str. 182, 50733 Köln",https://www.therapie.de/profil/nachtigall/,POINT (6.95492 50.95998)
1,Dipl.-Psych. Bettina Fökel,"Aduchtstr. 8, 50668 Köln",https://www.therapie.de/profil/foekel/,POINT (6.96517 50.95580)
2,Dr. Fariba Mohiyeddini,"Theodor-Heuss-Ring 58, 50668 Köln",https://www.therapie.de/profil/mohiyeddini/,POINT (6.96513 50.94978)
3,Annett Schlatmann,"Dönhoffstraße 32, 51373 Leverkusen",https://www.therapie.de/profil/schlatmann/,POINT (6.98503 51.03108)
4,Marion Gelhard,"Wiesdorfer Platz 23, 51373 Leverkusen",https://www.therapie.de/profil/marion.gelhard/,POINT (6.98547 51.03202)
5,Leander Vivian Dehm,"Hohenzollernring 49, 50672 Köln",https://www.therapie.de/profil/dehm/,POINT (6.93933 50.93956)
6,Saskia Tolman,"Hohenzollernring 28, 50672 Köln",https://www.therapie.de/profil/tolman/,POINT (6.93981 50.93841)
7,Dipl.-Psych. Marco Winter,"Gertrudenstraße 9, 50667 Köln",https://www.therapie.de/profil/marco.winter/,POINT (6.94556 50.93734)
8,Dr. Steffen Wolfsgruber,"Mittelstraße 16-18, 50672 Köln",https://www.therapie.de/profil/wolfsgruber/,POINT (6.94320 50.93704)
9,Dipl.-Psych. Hans Georg Nowak,"Cäcilienstr. 44, 50667 Köln",https://www.therapie.de/profil/nowak/,POINT (6.95054 50.93560)


In [56]:
data.explore()

In [46]:
gpd.tools.geocode(data.address)

,geometry,address
0,POINT (6.95492 50.95998),"182, Neusser Straße, 50733, Neusser Straße, Kö..."
1,POINT (6.96517 50.95580),"8, Aduchtstraße, 50668, Aduchtstraße, Köln, No..."
2,POINT (6.96513 50.94978),"58, Theodor-Heuss-Ring, 50668, Theodor-Heuss-R..."
3,POINT (6.98503 51.03108),"32, Dönhoffstraße, 51373, Dönhoffstraße, Lever..."
4,POINT (6.98547 51.03202),"23, Wiesdorfer Platz, 51373, Wiesdorfer Platz,..."
5,POINT (6.93933 50.93956),"49, Hohenzollernring, 50672, Hohenzollernring,..."
6,POINT (6.93981 50.93841),"28, Hohenzollernring, 50672, Hohenzollernring,..."
7,POINT (6.94556 50.93734),"9, Gertrudenstraße, 50667, Gertrudenstraße, Kö..."
8,POINT (6.94320 50.93704),"16-18, Mittelstraße, 50672, Mittelstraße, Köln..."
9,POINT (6.95054 50.93560),"Warhammer, 42-44, Cäcilienstraße, 50667, Cäcil..."
